In [1]:
%pip install -qU langchain-ollama
%pip install -U ollama
%pip install python-dotenv
%pip install google-search-results
%pip install numexpr
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import and load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
# from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2-vision",
    temperature = 0,
    # num_predict = 256,
    # other params ...
)

# llm = OllamaLLM(model="llama3.2-vision")

### Creating a LangSmith client

In [4]:
from langsmith import Client

client = Client()

In [5]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [6]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

### Initialize an agent

In [7]:
# Print available agent types
print(list(AgentType))

[<AgentType.ZERO_SHOT_REACT_DESCRIPTION: 'zero-shot-react-description'>, <AgentType.REACT_DOCSTORE: 'react-docstore'>, <AgentType.SELF_ASK_WITH_SEARCH: 'self-ask-with-search'>, <AgentType.CONVERSATIONAL_REACT_DESCRIPTION: 'conversational-react-description'>, <AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION: 'chat-zero-shot-react-description'>, <AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION: 'chat-conversational-react-description'>, <AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION: 'structured-chat-zero-shot-react-description'>, <AgentType.OPENAI_FUNCTIONS: 'openai-functions'>, <AgentType.OPENAI_MULTI_FUNCTIONS: 'openai-multi-functions'>]


In [8]:
agent = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

/var/folders/mv/5skqq8p925lbtt_m8d7msnkc0000gn/T/ipykernel_72043/595443142.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


### Input processing with exception handling

In [11]:
import nest_asyncio
import asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Define a list of input examples
inputs = [
    "How many go on vacation to Maldives in 2023?",
    "Who is the faster man in the World?",
    "What is Obama's first daughter's age?",
    "What is the distance of Africa to America?",
    "What was the number of games Lionel Messi played in Inter Miami?",
    "What was the total number of male in America?",
    "How many artistes performed at 02 Arena in London in 2023?",
    "What is 10 raised to power of 20?",
    "Who is the best international news station?",
    "What is value of 10000 divided by 2?",
]

results = []

async def run_agent(agent, input_example):
    try:
        return await agent.run(input_example)
    except Exception as e:
        # Handle exceptions, if any
        return e

# Create an event loop and gather results
async def main():
    loop = asyncio.get_event_loop()
    for input_example in inputs:
        results.append(loop.create_task(run_agent(agent, input_example)))
    await asyncio.gather(*results)

# Run the main event loop
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

/var/folders/mv/5skqq8p925lbtt_m8d7msnkc0000gn/T/ipykernel_72043/921604161.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return await agent.run(input_example)


In [12]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers
wait_for_all_tracers()

## Evaluating and testing AI applications using LangSmith

### Creating a LangSmith dataset

In [16]:
import uuid

# Generate a unique identity using UUID
unique_identity = str(uuid.uuid4())

dataset_name = f"calculator-example-dataset-{unique_identity}"

dataset = client.create_dataset(
    dataset_name, description="A calculator dataset"
)

runs = client.list_runs(
    project_name=os.environ["LANGCHAIN_PROJECT"],
    execution_order=1,
    error=False,
)
for run in runs:
    client.create_example(inputs=run.inputs, outputs=run.outputs, dataset_id=dataset.id)

### Initializing new agents

In [17]:
from langchain_ollama import ChatOllama
from langchain.agents import AgentType, initialize_agent, load_tools

llm = ChatOllama(model="mistral-nemo", temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

def agent_factory():
    return initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

### Customizing and configuring evaluation output

In [34]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [35]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama
from langchain.chat_models import ChatOpenAI

# Initialize the evaluation LLM
# evaluation_llm = OllamaLLM(model="llama3.2-vision")
evaluation_llm = ChatOpenAI(model="gpt-4o-mini")  # or replace with a supported model

evaluation_config = RunEvalConfig(

    evaluators=[

        EvaluatorType.QA,

        EvaluatorType.EMBEDDING_DISTANCE,

        RunEvalConfig.LabeledCriteria("helpfulness"),

        RunEvalConfig.Criteria(
            {
                "fifth-grader-score": "Do you have to be smarter than a fifth grader to answer this question?"
            }
        ),
    ],
    custom_evaluators=[],
    llm=evaluation_llm,  # Provide the evaluation LLM here
)

### Executing the agent and evaluator

In [36]:
from langchain.smith import (
    arun_on_dataset,
    run_on_dataset, 
)

chain_results = await arun_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_factory,
    evaluation=evaluation_config,
    verbose=True,
    tags=["testing-notebook"], 
)

/Users/Shatten/.pyenv/versions/3.11.2/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3506: LangChainPendingDeprecationWarning: The tags argument is deprecated and will be removed in a future release. Please specify project_metadata instead.
  await eval(code_obj, self.user_global_ns, self.user_ns)


View the evaluation results for project 'excellent-sweater-12' at:
https://eu.smith.langchain.com/o/d3451cd8-cf10-4e87-b058-be401896eaf6/datasets/58cf4193-9ef4-4ef4-8591-aa0a4bb358f7/compare?selectedSessions=ca3b2224-458b-4b2d-a01a-3dc1c9cb5f49

View all tests for Dataset calculator-example-dataset-1fa73a58-cda5-4249-921a-325fed3bcbd7 at:
https://eu.smith.langchain.com/o/d3451cd8-cf10-4e87-b058-be401896eaf6/datasets/58cf4193-9ef4-4ef4-8591-aa0a4bb358f7
[------------------->                              ] 4/10

Chain failed for example a6f75e18-aec7-4d55-a8a0-fb575cba68a4 with inputs {'input': 'What is 10 raised to power of 20?'}
ValueError('LLMMathChain._evaluate("\n10**20\n") raised error: Python int too large to convert to C long. Please try again with a valid numerical expression')


[------------------------------------------------->] 10/10

,feedback.correctness,feedback.embedding_cosine_distance,feedback.helpfulness,feedback.fifth-grader-score,error,execution_time,run_id
count,9.000000,9.000000,9.000000,9.000000,1,10.000000,10
unique,NaN,NaN,NaN,NaN,1,NaN,10
top,NaN,NaN,NaN,NaN,"LLMMathChain._evaluate(""\n10**20\n"") raised er...",NaN,84c0b0da-635b-4597-a294-f21528d10633
freq,NaN,NaN,NaN,NaN,1,NaN,1
mean,0.222222,0.242218,0.666667,0.666667,NaN,53.013710,NaN
std,0.440959,0.049401,0.500000,0.500000,NaN,22.220173,NaN
min,0.000000,0.186911,0.000000,0.000000,NaN,26.079506,NaN
25%,0.000000,0.207105,0.000000,0.000000,NaN,40.248332,NaN
50%,0.000000,0.231350,1.000000,1.000000,NaN,47.747456,NaN
75%,0.000000,0.247677,1.000000,1.000000,NaN,53.750314,NaN
